In [ ]:
from pathlib import Path

import pandas as pd # type: ignore

import src.balance_tracker_pipeline_v2
import src.cdutils.database.sliding_window
import src.excel_output
import src.monthly_delta
from src._version import __version__

# Fetch Data from COCC
data_prior, data_current = src.cdutils.database.sliding_window.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)

In [ ]:
def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df['Net Balance'] * df['noteintrate']
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x['Net Balance'].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

In [ ]:
import numpy as np

In [ ]:
def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

    Args:
        df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

    Returns:
        df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
    """
    # Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
    df = df.copy()
    
    df['noteintrate'] = np.where(
        (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
        np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
        df['noteintrate']))))))))))))))
    )

    # Create total yield
    total_yield = weighted_avg_rate(df, title='Total Loan Yield')

    # Create new loan yield
    datetime_cols = ['effdate','origdate']
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col])

    new_loan_df = df[
        (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
    ].copy()

    new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

    df['Unadvanced'] = df['Total Exposure'] - df['Net Balance']

    # Unadvanced
    unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
    unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

    # Merge
    merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
    merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

    return merged_df




In [ ]:
merged_df = yield_and_unadvanced_creation(full_data_current)

In [ ]:
merged_df

In [ ]:

import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd

def fetch_from_acctuserfield():
    """
    Gets data from COCC
    """
    wh_acctuserfields = text(f"""
    SELECT
        *
    FROM 
        OSIBANK.WH_ACCTUSERFIELDS a
    """)

    queries = [
        {'key':'wh_acctuserfields', 'sql':wh_acctuserfields, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [ ]:
acctuser = fetch_from_acctuserfield()

In [ ]:
acctuser = acctuser['wh_acctuserfields'].copy()

In [ ]:
acctuser

In [ ]:
splt = acctuser[acctuser['acctuserfieldcd'] == 'SPLT'].copy()

In [ ]:
assert splt['acctnbr'].is_unique, "Failure"

In [ ]:
"""
Using the lookup query to inspect the DB tables
"""

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data():
    """
    Main data query
    """
    # Engine 1
    lookup_df = text("""
    SELECT 
        *
    FROM 
        sys.all_tab_columns col
    """)

    queries = [
        # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
        {'key':'lookup_df', 'sql':lookup_df, 'engine':2},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
lookup = fetch_data()

In [ ]:
lookup = lookup['lookup_df'].copy()

In [ ]:
lookup

In [ ]:

import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd
from datetime import date, datetime
# ------------------------------------------------------------------
# --- 1.  Runtime inputs -------------------------------------------
# ------------------------------------------------------------------
begin_dt: datetime = datetime(2024, 3, 1)   # ← change as you like
end_dt:   datetime = datetime(2024, 3, 31)
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
secondary_codes   = ["DDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]
# ------------------------------------------------------------------
# --- 2.  Helpers ---------------------------------------------------
# ------------------------------------------------------------------
def sql_list(py_list):
    """Turn ['A','B'] →  'A','B'  for SQL IN (...)"""
    return ", ".join(f"'{x}'" for x in py_list)
def to_date_literal(dt: datetime) -> str:
    """
    Return an Oracle-friendly TO_DATE literal.
      2024-03-15  →  TO_DATE('2024-03-15','YYYY-MM-DD')
    If the value has a time component, include it:
      2024-03-15 14:07:00  →  TO_DATE('2024-03-15 14:07:00','YYYY-MM-DD HH24:MI:SS')
    """
    fmt_date = "%Y-%m-%d %H:%M:%S" if isinstance(dt, datetime) and dt.time() != datetime.min.time() else "%Y-%m-%d"
    date_str = dt.strftime(fmt_date)
    mask     = "YYYY-MM-DD HH24:MI:SS" if " " in date_str else "YYYY-MM-DD"
    return f"TO_DATE('{date_str}','{mask}')"
# Pre-formatted pieces
resi_minors_sql        = sql_list(resi_minors)
secondary_codes_sql    = sql_list(secondary_codes)
cml_minors_sql         = sql_list(cml_minors)
disb_codes_cml_sql     = sql_list(disb_codes_cml)
receipt_codes_cml_sql  = sql_list(receipt_codes_cml)
start_date = "2025-03-01 00:00:00"
end_date = "2025-03-31 00:00:00"
# ------------------------------------------------------------------
# --- 3.  Compose the SQL ------------------------------------------
# ------------------------------------------------------------------

def fetch_from_database():
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [ ]:
data = fetch_from_database()

In [ ]:
rtxn = data['rtxn'].copy()

In [ ]:
rtxn

In [ ]:
def fetch_from_database():
    """
    Gets data from COCC
    """
    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        OSIBANK.WH_ACCTCOMMON a
    """)

    queries = [
        {'key':'acctcommon', 'sql':acctcommon, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
data = fetch_from_database()

In [ ]:
ac = data['acctcommon'].copy()

In [ ]:
import cdutils.input_cleansing # type: ignore

rtxn_schema = {
    'acctnbr': str
}

rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

ac_schema = {
    'acctnbr': str
}
ac = cdutils.input_cleansing.enforce_schema(ac, ac_schema)


In [ ]:
rtxn = pd.merge(rtxn, ac, how='left', on='acctnbr')

In [ ]:
rtxn

In [ ]:
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
resi_codes        = ["PDSB","CWTH","CKUS","XDSB"]
secondary_codes   = ["PDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]

In [ ]:
import numpy as np

In [ ]:
rtxn['adv_calc'] = abs(np.where((rtxn['currmiaccttypcd'].isin(resi_minors)) and (rtxn['tranamt'] < 0) and (rtxn['trantypcd'].isin(resi_codes)), rtxn['tranamt'],
                            (np.where(rtxn['trantypcd'].isin(secondary_codes) and (~(rtxn['currmiaccttypcd']).isin(cml_minors))), rtxn['tranamt'],
                            0)))

In [ ]:
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v2
import src.cdutils.database.sliding_window
import src.excel_output
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.cdutils.database.sliding_window.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v2.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)

In [ ]:
df = full_data_current.copy()

In [ ]:
# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
"""
Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

Args:
    df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

Returns:
    df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
"""
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = df.copy()

# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate']))))))))))))))
)


In [ ]:

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
print(df.info())
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# # Attach transaction amt

# # Create advances column
# resi_minor_
# new_loan_df['Advances'] = np.where(
#     ()
# )

new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

return merged_df


In [ ]:
additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)


In [ ]:


TEMPLATE_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
# OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD_test.xlsx')

src.excel_output.update_excel_template(TEMPLATE_PATH, monthly_delta, additional_fields, OUTPUT_PATH)

# Distribution 
recipients = [
    "chad.doorley@bcsbmail.com"
    # "Timothy.Chaves@bcsbmail.com",
    # "John.Silva@bcsbmail.com",
    # "Dawn.Young@bcsbmail.com",
    # "Christopher.Alves@bcsbmail.com",
    # "donna.oliveira@bcsbmail.com",
    # "nancy.pimentel@bcsbmail.com",
    # "Hasan.Ali@bcsbmail.com",
    # "Michael.Patacao@bcsbmail.com",
    # "Jeffrey.Pagliuca@bcsbmail.com",
    # "Erin.Riendeau@bcsbmail.com",
    # "donna.pavao@bcsbmail.com"
]
bcc_recipients = [
    "chad.doorley@bcsbmail.com",
    "businessintelligence@bcsbmail.com"
]
subject = f"Balance Tracker YTD - Through April 2025" 
body = "Hi all, \n\nAttached is the Balance Tracker through the most recent month end. If you have any questions, please reach out to BusinessIntelligence@bcsbmail.com\n"
attachment_paths = [OUTPUT_PATH]

cdutils.distribution.email_out(
    recipients = recipients, 
    bcc_recipients = bcc_recipients, 
    subject = subject, 
    body = body, 
    attachment_paths = attachment_paths
    )



if __name__ == '__main__':
print(f"Starting {__version__}")
main()
print("Complete!")



In [ ]:
import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
import cdutils.input_cleansing # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd

def fetch_from_acctuserfield():
        """
        Gets data from COCC
        """
        wh_acctuserfields = text(f"""
        SELECT
            *
        FROM 
            OSIBANK.WH_ACCTUSERFIELDS a
        """)

        queries = [
            {'key':'wh_acctuserfields', 'sql':wh_acctuserfields, 'engine':1},
        ]

        data = cdutils.database.connect.retrieve_data(queries)
        return data


# def append_dealersplit(df: pd.DataFrame):
"""
Attach secondary lending officer to any dataframe
"""

data = fetch_from_acctuserfield()

wh_acctuserfields = data['wh_acctuserfields'].copy()


In [ ]:
wh_acctuserfields

In [ ]:
splt = wh_acctuserfields[wh_acctuserfields['acctuserfieldcd'] == 'SPLT'].copy()

splt = splt.sort_values(by='datelastmaint', ascending=False).copy()
splt = cdutils.deduplication.dedupe([{'df':splt, 'field':'acctnbr'}])


# Asserts
assert splt['acctnbr'].is_unique, "splt not unique on acctnbr"


splt = splt[['acctnbr','acctuserfieldvalue']].copy()

splt = splt.rename(columns={'acctuserfieldvalue':'SPLT'}).copy()



In [ ]:
splt

In [ ]:
splt.info()

In [ ]:
splt['SPLT'] = pd.to_numeric(splt['SPLT'], errors="coerce").fillna(0.0)


In [ ]:


schema_df = {
            'acctnbr': str,
        }

df = cdutils.input_cleansing.enforce_schema(df, schema_df)

schema_splt = {
            'acctnbr': str,
            'SPLT': float
        }

splt = cdutils.input_cleansing.enforce_schema(splt, schema_splt)

assert df['acctnbr'].is_unique, "acctnbr not unique in df"

final_df = pd.merge(df, splt, on='acctnbr', how='left')

# return final_df




In [ ]:
final_df

In [ ]:
final_df['SPLT'] = pd.to_numeric(final_df['SPLT'], errors="coerce").fillna(0.0)


In [ ]:
final_df

In [ ]:
final_df[final_df['Category'].isnull()]

In [ ]:
### 2025-05-09
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v3
import src.excel_output
import src.fetch_data
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.fetch_data.fetch_data()



In [ ]:
_, data_prior_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)
additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)


#     TEMPLATE_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
#     OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD.xlsx')
#     # OUTPUT_PATH = Path('./output/Portfolio_Balance_Tracker_2025YTD_test.xlsx')

#     src.excel_output.update_excel_template(TEMPLATE_PATH, monthly_delta, additional_fields, OUTPUT_PATH)

#     # Distribution 
#     recipients = [
#         # "chad.doorley@bcsbmail.com"
#         "Timothy.Chaves@bcsbmail.com",
#         "John.Silva@bcsbmail.com",
#         "Dawn.Young@bcsbmail.com",
#         "Christopher.Alves@bcsbmail.com",
#         "donna.oliveira@bcsbmail.com",
#         "nancy.pimentel@bcsbmail.com",
#         "Hasan.Ali@bcsbmail.com",
#         "Michael.Patacao@bcsbmail.com",
#         "Jeffrey.Pagliuca@bcsbmail.com",
#         "Erin.Riendeau@bcsbmail.com",
#         "donna.pavao@bcsbmail.com"
#     ]
#     bcc_recipients = [
#         "chad.doorley@bcsbmail.com",
#         "businessintelligence@bcsbmail.com"
#     ]
#     subject = f"Balance Tracker YTD - Through April 2025" 
#     body = "Hi all, \n\nAttached is the Balance Tracker through the most recent month end. If you have any questions, please reach out to BusinessIntelligence@bcsbmail.com\n\n"
#     attachment_paths = [OUTPUT_PATH]

#     cdutils.distribution.email_out(
#         recipients = recipients, 
#         bcc_recipients = bcc_recipients, 
#         subject = subject, 
#         body = body, 
#         attachment_paths = attachment_paths
#         )



# if __name__ == '__main__':
#     print(f"Starting {__version__}")
#     main()
#     print("Complete!")



In [ ]:
full_data_current

In [ ]:
monthly_delta

In [ ]:
additional_fields

In [ ]:
df = full_data_current.copy()

In [ ]:
"""
Weighted Average Rate & Unadvanced Funds
"""
import cdutils.dealer_split # type: ignore

import pandas as pd # type: ignore
import numpy as np # type: ignore


def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate', weight_col: str = 'Net Balance', value_col: str = 'modified_noteintrate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df[weight_col] * df[value_col]
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x[weight_col].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

#     Args:
#         df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

#     Returns:
#         df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
#     """
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = df.copy()

# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate'])))))))))))))
)

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# # Attach transaction amt

# # Create advances column
# resi_minor_
# new_loan_df['Advances'] = np.where(
#     ()
# )




In [ ]:
new_loan_df

In [ ]:
slice = new_loan_df[new_loan_df['currmiaccttypcd'] == "CM06"].copy()

In [ ]:
slice

In [ ]:
# Here we need to create a new column that would be the new balance + advances

In [ ]:
start_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-01 00:00:00')
end_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-%d 00:00:00')


In [ ]:
from sqlalchemy import text # type: ignore


In [ ]:
def fetch_rtxn():
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
    """)

    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        OSIBANK.WH_ACCTCOMMON a
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
        {'key':'acctcommon', 'sql':acctcommon, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [ ]:
rtxn_data_pack = fetch_rtxn()

In [ ]:
acctcommon = rtxn_data_pack['acctcommon'].copy()
rtxn = rtxn_data_pack['rtxn'].copy()


In [ ]:
rtxn_schema = {'tranamt': 'float', 'acctnbr':'str'}
rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

acctcommon_schema = {'acctnbr':'str'}
acctcommon = cdutils.input_cleansing.enforce_schema(acctcommon, acctcommon_schema)


In [ ]:
rtxn = pd.merge(rtxn, acctcommon, how='left', on='acctnbr')

In [ ]:
rtxn

In [ ]:
"""
resi_minors       = ["MG48", "MG50", "MG52", "MG55", "MG60"]
resi_codes        = ["PDSB","CWTH","CKUS","XDSB"]
secondary_codes   = ["PDSB", "OPA"]
cml_minors        = ["CM06", "CM30", "CM52"]
disb_codes_cml    = ["PDSB", "SWPI"]
receipt_codes_cml = ["PRCT", "SWPR"]
"""
resi_minors = ("MG48", "MG50", "MG52", "MG55", "MG60")
resi_codes = ("PDSB","CWTH","CKUS","XDSB")
secondary_codes = ("PDSB", "OPA")
cml_minors = ("CM06", "CM30", "CM52")
disb_codes_cml = ("PDSB", "SWPI")
receipt_codes_cml = ("PRCT", "SWPR")


In [ ]:
def calculate_net_advances(row):
    if (row['currmiaccttypcd'] in resi_minors) and (row['tranamt'] < 0) and (row['rtxntypcd'] in resi_codes):
        return row['tranamt']
    elif (row['rtxntypcd'] in secondary_codes) and (row['currmiaccttypcd'] not in cml_minors):
        return row['tranamt']
    else:
        return 0

In [ ]:
rtxn['net advances'] = rtxn.apply(calculate_net_advances, axis=1)

In [ ]:
# rtxn['net advances'] = abs(rtxn['net advances'])

In [ ]:
rtxn.info()

In [ ]:
minor_df = rtxn.groupby('currmiaccttypcd')['net advances'].sum().reset_index()
minor_df['net advances'] = abs(minor_df['net advances'])
minor_df = minor_df.sort_values(by='net advances', ascending=False)
minor_df

In [ ]:
pdsb = rtxn[rtxn['rtxntypcd'] == 'PDSB'].copy()

In [ ]:
pdsb.info()

In [ ]:
pdsb.describe()

In [ ]:

new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

    # return merged_df


In [ ]:
"""
Balance Tracker - revised
Developed by CD
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.addition_fields
import src.balance_tracker_pipeline_v3
import src.excel_output
import src.fetch_data
import src.monthly_delta
from src._version import __version__
import cdutils.distribution # type: ignore

# def main():
# Fetch Data from COCC
data_prior, data_current = src.fetch_data.fetch_data()

_, data_prior_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_prior)
full_data_current, data_current_summary = src.balance_tracker_pipeline_v3.main_pipeline_bt(data_current)

# OUTPUT_PATH = Path('./output/data_prior_summary.xlsx')
# data_prior_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

# OUTPUT_PATH = Path('./output/data_current_summary.xlsx')
# data_current_summary.to_excel(OUTPUT_PATH, engine='openpyxl', index=False)

monthly_delta = src.monthly_delta.creating_monthly_delta(data_prior_summary, data_current_summary)
# additional_fields = src.addition_fields.yield_and_unadvanced_creation(full_data_current)

In [ ]:
"""
Weighted Average Rate & Unadvanced Funds
"""
import cdutils.dealer_split # type: ignore
import src.fetch_data

import pandas as pd # type: ignore
import numpy as np # type: ignore


def weighted_avg_rate(df: pd.DataFrame, title: str='Weighted Avg Rate', weight_col: str = 'Net Balance', value_col: str = 'modified_noteintrate') -> pd.DataFrame:
    """
    Create weighted average rate
    """
    df = df.copy()
    df['WeightedRate'] = df[weight_col] * df[value_col]
    grouped_df = df.groupby('Category').apply(
        lambda x: x['WeightedRate'].sum() / x[weight_col].sum(), include_groups=False
    ).reset_index(name=title).copy()
    return grouped_df

# def yield_and_unadvanced_creation(df: pd.DataFrame) -> pd.DataFrame:
"""
Takes the full data current (from the main pipeline) and creates a total loan yield, new loan yield, and unadvanced funds for every category

Args:
    df (pd.Dataframe): This is the full_data_current from the current output of the pipeline function

Returns:
    df (pd.DataFrame): Simple table that will be added to the right most columns of the balance tracker in a separate excel update function
"""
# Manual Rate Adjustments for Heat Loans (CNS) WSJ + 100 bp
df = full_data_current.copy()

In [ ]:
# Attempting to make this 1 query. Takes more work than expected
# import datetime
# from typing import Dict

# from sqlalchemy import text # type: ignore

# import cdutils.database.connect # Type: ignore

# def fetch_rtxn(start_date, end_date):
#     """
#     Gets data from COCC
#     """
#     rtxn = text(f"""
#     SELECT
#         t.RTXNNBR,
#         t.RTXNTYPCD,
#         t.ACCTNBR,
#         t.TRANAMT,
#         a.CURRMIACCTTYPCD
#     FROM
#         COCCDM.WH_RTXN t
#     JOIN
#         COCCDM.WH_ACCTCOMMON a
#         ON t.ACCTNBR = a.ACCTNBR
#         AND t.RUNDATE = a.EFFDATE
#     WHERE
#         (a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')) AND
#         (a.CURRRTXNSTATCD = 'C')
#     """)

#     acctcommon = text(f"""
#     SELECT
#         a.ACCTNBR,
#         a.CURRMIACCTTYPCD
#     FROM
#         COCCDM.WH_ACCTCOMMON_ME a
#     """)

#     queries = [
#         {'key':'rtxn', 'sql':rtxn, 'engine':2},
#         {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
#     ]

#     data = cdutils.database.connect.retrieve_data(queries)
#     return data

In [ ]:

import datetime
from typing import Dict

from sqlalchemy import text # type: ignore

import cdutils.database.connect # Type: ignore

def fetch_rtxn(start_date, end_date):
    """
    Gets data from COCC
    """
    rtxn = text(f"""
    SELECT
        a.RTXNNBR,
        a.RTXNTYPCD,
        a.ACCTNBR,
        a.TRANAMT
    FROM
        COCCDM.WH_RTXN a
    WHERE
        (a.RUNDATE BETWEEN TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')) AND
        (a.CURRRTXNSTATCD = 'C')
    """)

    acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.CURRMIACCTTYPCD
    FROM
        COCCDM.WH_ACCTCOMMON_ME a
    """)

    queries = [
        {'key':'rtxn', 'sql':rtxn, 'engine':2},
        {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [ ]:


# Adjustment for the Consumer loans: WSJ Prime + 1
df['modified_noteintrate'] = np.where(
    (df['currmiaccttypcd'] == 'IL33') & (df['contractdate'] >= pd.Timestamp(2025,1,1)), .07,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,12,19)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,11,8)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2024,9,19)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,7,27)), .095,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,5,4)), .0925,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,3,23)), .09,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2023,2,2)), .0875,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,12,16)), .085,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,11,3)), .08,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,9,22)), .0725,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,7,28)), .065,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])) & (df['contractdate'] >= pd.Timestamp(2022,6,16)), .0575,
    np.where((df['currmiaccttypcd'].isin(['IL21','IL31','IL33'])), .05,
    df['noteintrate'])))))))))))))
)

# Attach dealer split rate and subtract this from noteint rate
df = cdutils.dealer_split.append_dealersplit(df)
df['modified_noteintrate'] = np.where(df['Category'] == 'Indirect', df['modified_noteintrate'] - df['SPLT'], df['modified_noteintrate'])



# Create total yield
total_yield = weighted_avg_rate(df, title='Total Loan Yield')

# Create new loan yield
datetime_cols = ['effdate','origdate']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col])

new_loan_df = df[
    (df['origdate'].dt.month == df['effdate'].dt.month) & (df['origdate'].dt.year == df['effdate'].dt.year)
].copy()

# Using the Transaction table
start_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-01 00:00:00')
end_date = new_loan_df['effdate'].iloc[0].strftime('%Y-%m-%d 00:00:00')

# rtxn_data_pack = fetch_rtxn(start_date, end_date)

acctcommon = rtxn_data_pack['acctcommon'].copy()
rtxn = rtxn_data_pack['rtxn'].copy() 
rtxn_schema = {'tranamt': 'float', 'acctnbr':'str'}
rtxn = cdutils.input_cleansing.enforce_schema(rtxn, rtxn_schema)

acctcommon_schema = {'acctnbr':'str'}
acctcommon = cdutils.input_cleansing.enforce_schema(acctcommon, acctcommon_schema)
rtxn = pd.merge(rtxn, acctcommon, how='left', on='acctnbr')

# Calculting net advances
resi_minors = ("MG48", "MG50", "MG52", "MG55", "MG60")
resi_codes = ("PDSB","CWTH","CKUS","XDSB")
secondary_codes = ("PDSB", "OPA")
cml_minors = ("CM06", "CM30", "CM52")
disb_codes_cml = ("PDSB", "SWPI")
receipt_codes_cml = ("PRCT", "SWPR")

def calculate_advances(row):
    if (row['currmiaccttypcd'] in resi_minors) and (row['tranamt'] < 0) and (row['rtxntypcd'] in resi_codes):
        return row['tranamt']
    elif (row['rtxntypcd'] in secondary_codes) and (row['currmiaccttypcd'] not in cml_minors):
        return row['tranamt']
    else:
        return 0

rtxn['advances'] = rtxn.apply(calculate_advances, axis=1)

# Here we'll group by acctnbr and sum net advances
# Apply abs value to net advances

# Create a separate formula on transaction table for the 3 commercial minors that we need to net disb - reciepts
# or embed this into the second line of that formula above (couldn't get this correct based on the formula/codes Tom gave me)

# Then we merge it into new_load_df as an extra field and this is used in the weighted_avg_rate as the weight_col instead of Net Balance






In [ ]:
# Need trancd formula built out: NDSB
# This will be a separate column that we add to advances
rtxn['new loan disb'] = np.where(rtxn['rtxntypcd'] == 'NDSB', rtxn['tranamt'], 0)

In [ ]:
# CML
cml_minors = ("CM06", "CM30", "CM52")
# Group by acctnbr
disb_codes_cml = ("PDSB", "SWPI") # Increasing the loan
receipt_codes_cml = ("PRCT", "SWPR") # Paying down loan
# If positive, disb - receipt
# Else 0


In [ ]:
rtxn['cml_disb'] = np.where((rtxn['rtxntypcd'].isin(disb_codes_cml)) & (rtxn['currmiaccttypcd'].isin(cml_minors)), rtxn['tranamt'], 0)
rtxn['cml_receipt'] = np.where((rtxn['rtxntypcd'].isin(receipt_codes_cml)) & (rtxn['currmiaccttypcd'].isin(cml_minors)), rtxn['tranamt'], 0)


In [ ]:
acct_grouping = rtxn.groupby('acctnbr').agg({
    'advances':'sum',
    'new loan disb':'sum',
    'cml_disb':'sum',
    'cml_receipt':'sum'
}).reset_index()

In [ ]:
acct_grouping

In [ ]:
acct_grouping['advances'] = abs(acct_grouping['advances'])
acct_grouping['net cml advance'] = (acct_grouping['cml_disb'] + acct_grouping['cml_receipt']) * -1
acct_grouping['net cml advance'] = acct_grouping['net cml advance'].replace(-0.0,0.0)
acct_grouping['net cml advance'] = np.where(acct_grouping['net cml advance'] < 0, 0, acct_grouping['net cml advance'])
acct_grouping['advances'] = np.where(acct_grouping['net cml advance'] > 0, acct_grouping['net cml advance'], acct_grouping['advances'])
acct_grouping_final = acct_grouping[['acctnbr','advances','new loan disb']].copy()

# Merging
new_loan_df = pd.merge(df, acct_grouping_final, on='acctnbr', how='left')


In [ ]:
new_loan_df['advances'] = new_loan_df['advances'].fillna(0)
new_loan_df['new loan disb'] = new_loan_df['new loan disb'].fillna(0)
new_loan_df['new_and_advanced'] = new_loan_df['advances'] + (new_loan_df['new loan disb'] * -1)

In [ ]:
new_loan_df

In [ ]:
ytd_table_recon = new_loan_df.groupby('currmiaccttypcd')['new_and_advanced'].sum().sort_values(ascending=False).reset_index()

In [ ]:
ytd_table_recon6

In [ ]:
# minor_df = rtxn.groupby('currmiaccttypcd')['net advances'].sum().reset_index()
# minor_df['net advances'] = abs(minor_df['net advances'])
# minor_df = minor_df.sort_values(by='net advances', ascending=False)
# minor_df

In [ ]:
new_yield = weighted_avg_rate(new_loan_df, title='New Loan Yield', weight_col='new_and_advanced')

df['Unadvanced'] = (df['availbalamt']) / 1000

# Unadvanced
unadvanced = df.groupby('Category')['Unadvanced'].sum().reset_index()
unadvanced = unadvanced.rename(columns={'Unadvanced':'Unadvanced Funds'})

# Merge
merged_df = pd.merge(new_yield, total_yield, how='inner', on='Category')
merged_df = pd.merge(merged_df, unadvanced, how='inner', on='Category')

# return merged_df

In [ ]:
merged_df